In [145]:
data = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_small.csv', engine='python')
data = data.sample(frac=1)

In [150]:
train = data[:3500]
test = data[3500:]

In [154]:
train.to_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', encoding='utf-8', index=False)
test.to_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', encoding='utf-8', index=False)

In [1]:
from collections import defaultdict
import time
import gc
import pandas as pd
import numpy as np

from tqdm import tqdm

s = time.perf_counter()
train = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', engine='python')
test = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', engine='python')

def combine(data):
    item_users = dict()
    user_items = dict()
    users = set()
    data['userId'] = 'user_' + data['userId'].astype(str)
    data['movieId'] = 'movie_' + data['movieId'].astype(str)
    for u, item, r in data[['userId', 'movieId', 'rating']].values:
        item_users.setdefault(item, []).append((u, r))
        user_items.setdefault(u, []).append((item, r))
        users.add(u)
    del data
    gc.collect()
    return item_users, user_items, users

train_item_users, train_user_items, train_users = combine(train)
test_item_users, test_user_items, test_users = combine(test)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

cost time 0.0018360216666666667 min


In [98]:
# from joblib import dump, load
# dump(item_users, r'D:\Users\hao.guo\比赛代码提炼\推荐系统\movielen\data\item_users.pkl')
# dump(users, r'D:\Users\hao.guo\比赛代码提炼\推荐系统\movielen\data\users.pkl')

In [2]:
import math
from tqdm import tqdm_notebook
def UserSimilarity(item_users, users):
    # build inverse table for item_users
#     item_users = dict()
#     users = set()
#     for u, items in data.items():
#         for i in items.keys():
#             # if i not in item_users:
#             #     item_users[i] = set()
#             item_users.setdefault(i, set()).add(u)
#         users.add(u)
    #calculate co-rated items between users
    # C = dict()
    C = defaultdict(dict)
    # C = np.zeros((len(item_users), len(item_users)))
    # user items list
    N = defaultdict(int)
    print('共现矩阵计算')
    for i, pairs in tqdm(item_users.items()):
        for u, r1 in pairs:
            N[u] += r1 ** 2
            for v, r2 in pairs:
                if u == v:
                    continue
                if v not in list(C[u]):
                    C[u][v] = 0
                C[u][v] += (r1 * r2 / np.log1p(len(pairs)))
    #calculate finial similarity matrix W
    W = defaultdict(dict)
    print('相似矩阵计算')
    for u, related_users in tqdm(C.items()):
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W
s = time.perf_counter()
W = UserSimilarity(train_item_users, train_users)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

共现矩阵计算


100%|███████████████████████████████████████████████████████████████████████████| 1694/1694 [00:00<00:00, 19523.13it/s]


相似矩阵计算


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 16693.08it/s]


cost time 0.002101749999999984 min


In [50]:
sorted(W['user_1'].items(), key=lambda p: p[1], reverse=True)[0:5]

[('user_21', 0.19303695393600245),
 ('user_3', 0.19136694720294242),
 ('user_25', 0.1847717426269238),
 ('user_24', 0.13473512787501885),
 ('user_47', 0.11828936840773875)]

In [5]:
def Recommend(user, train, W, K):
    rank = defaultdict(int)
    sim_sums = defaultdict(int)
    
    interacted_items = list(map(lambda p: p[0], train[user]))
    for v, wuv in sorted(W[user].items(), key=lambda p: p[1], reverse=True)[0:K]:
        for i, rvi in train[v]:
            if i in interacted_items:
            #we should filter items user interacted before
                continue
            rank[i] += wuv * rvi
            sim_sums[i] += wuv
    res = [(sim / 1, i) for i, sim in rank.items()]
    res.sort(reverse=True)
    ranked_items = [x[1] for x in res]
#     sorted(rank.items(), key=lambda p: p[1], reverse=True)
    return ranked_items[:50]

def Precision(train, test, W, N):
    hit = 0
    all = 0
    for user in test.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item in rank:
            if item in tu:
                hit += 1
        all += 50
    return hit / (all * 1.0)

def Recall(train, test, W, N):
    hit = 0
    all = 0
    for user in test.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)

def Coverage(train, test, W, N):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item, r in test[user]:
            all_items.add(item)
    for user in test.keys():
        rank = Recommend(user, train, W, N)
        for item in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)

def Popularity(train, test, W, N):
    item_popularity = dict()
    for user, items in train.items():
        for item, r in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in test.keys():
        rank = Recommend(user, train, W, N)
        for item in rank:
            ret += np.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret 

def eval(train_user_items, test_user_items, W, N):
    p = Precision(train_user_items, test_user_items, W, N)
    r = Recall(train_user_items, test_user_items, W, N)
    c = Coverage(train_user_items, test_user_items, W, N)
    pop = Popularity(train_user_items, test_user_items, W, N)
    return (p, r, c, pop)
    
# print(eval(train_user_items, test_user_items, W, 5))
print(eval(train_user_items, test_user_items, W, 10))
# print(eval(train_user_items, test_user_items, W, 20))
# print(eval(train_user_items, test_user_items, W, 30))
# print(eval(train_user_items, test_user_items, W, 40))
# print(eval(train_user_items, test_user_items, W, 50))

(0.1328, 0.22148098732488325, 0.37422680412371134, 2.0557740410362264)


In [58]:
(0.1196, 0.19946631087391595, 0.534020618556701, 1.901404008414894)
(0.1316, 0.21947965310206805, 0.35670103092783506, 2.0821030298441956)
(0.1368, 0.22815210140093395, 0.25876288659793817, 2.2046577786199233)
(0.136, 0.2268178785857238, 0.22783505154639175, 2.264661699694442)
(0.1352, 0.22548365577051369, 0.2134020618556701, 2.2836800705995213)
(0.134, 0.22348232154769845, 0.2134020618556701, 2.2868977375518935)

(299, 2500)

In [19]:
from collections import defaultdict
import time
import gc
import pandas as pd
import numpy as np

from tqdm import tqdm

s = time.perf_counter()
train = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', engine='python')
test = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', engine='python')

def combine(data):
    item_users = dict()
    user_items = dict()
    users = set()
    data['userId'] = 'user_' + data['userId'].astype(str)
    data['movieId'] = 'movie_' + data['movieId'].astype(str)
    for u, item, r in data[['userId', 'movieId', 'rating']].values:
        item_users.setdefault(item, []).append((u, r))
        user_items.setdefault(u, []).append((item, r))
        users.add(u)
    del data
    gc.collect()
    return item_users, user_items, users

train_item_users, train_user_items, train_users = combine(train)
test_item_users, test_user_items, test_users = combine(test)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

import math
from tqdm import tqdm_notebook

def ItemSimilarity(user_items):
    # build inverse table for user_items
    C = defaultdict(dict)

    # item users list
    N = defaultdict(int)
    print('共现矩阵计算')
    for u, pairs in tqdm(user_items.items()):
        for i, r1 in pairs:
            N[i] += r1 ** 2
            for j, r2 in pairs:
                if i == j:
                    continue
                if j not in list(C[i]):
                    C[i][j] = 0
                C[i][j] += (r1 * r2 / np.log1p(len(pairs)))
                
    #calculate finial similarity matrix W
    W = defaultdict(dict)
    print('相似矩阵计算')
    for i, related_items in tqdm(C.items()):
        for j, cij in related_items.items():
            W[i][j] = cij / math.sqrt(N[i] * N[j])
            
            
    # 归一化
    for k, v in W.items():
        max_w = np.max(list(v.values()))
        for j, w in v.items():
            v[j] = w / max_w
            
    return W
s = time.perf_counter()
W = ItemSimilarity(train_user_items)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

cost time 0.0021449133333362623 min
共现矩阵计算


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.56it/s]


相似矩阵计算


100%|████████████████████████████████████████████████████████████████████████████| 1694/1694 [00:00<00:00, 4981.04it/s]


cost time 0.08108804666666553 min


In [9]:
sorted(W['movie_1'].items(), key=lambda p: p[1], reverse=True)[0:5]

[('movie_1073', 0.5617167103983758),
 ('movie_95', 0.5596002151849775),
 ('movie_802', 0.5374641747711849),
 ('movie_786', 0.5222935341726456),
 ('movie_784', 0.49495500224376193)]

In [10]:
Recommend('user_1', train_user_items, W, 5)

['movie_2567',
 'movie_45499',
 'movie_1179',
 'movie_1127',
 'movie_2011',
 'movie_6003',
 'movie_47044',
 'movie_57640',
 'movie_54001',
 'movie_53464',
 'movie_3098',
 'movie_2428',
 'movie_231',
 'movie_150',
 'movie_39',
 'movie_316',
 'movie_296',
 'movie_50872',
 'movie_40339',
 'movie_52287',
 'movie_1276',
 'movie_4886',
 'movie_7373',
 'movie_56775',
 'movie_55995',
 'movie_53125',
 'movie_3826',
 'movie_4235',
 'movie_6863',
 'movie_2333',
 'movie_1894',
 'movie_3827',
 'movie_952',
 'movie_1302',
 'movie_3408',
 'movie_54997',
 'movie_919',
 'movie_468',
 'movie_1210',
 'movie_59376',
 'movie_920',
 'movie_1449',
 'movie_6942',
 'movie_3004',
 'movie_852',
 'movie_1653',
 'movie_2881',
 'movie_1097',
 'movie_1183',
 'movie_35836']

In [20]:
def Recommend(user, train, W, K):
    rank = defaultdict(int)
    
    interacted_items = list(map(lambda p: p[0], train[user]))
    for item, rji in train[user]:
        for j, wij in sorted(W[item].items(), key=lambda p: p[1], reverse=True)[0:K]:
            if j in interacted_items:
            #we should filter items user interacted before
                continue
            rank[j] += wij * rji

    res = sorted(rank.items(), key=lambda p: p[1], reverse=True)
    ranked_items = [x[0] for x in res]
    return ranked_items[:50]

def Precision(train, test, W, N):
    hit = 0
    all = 0
    for user in test.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item in rank:
            if item in tu:
                hit += 1
        all += 50
    return hit / (all * 1.0)

def Recall(train, test, W, N):
    hit = 0
    all = 0
    for user in test.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)

def Coverage(train, test, W, N):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item, r in test[user]:
            all_items.add(item)
    for user in test.keys():
        rank = Recommend(user, train, W, N)
        for item in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)

def Popularity(train, test, W, N):
    item_popularity = dict()
    for user, items in train.items():
        for item, r in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in test.keys():
        rank = Recommend(user, train, W, N)
        for item in rank:
            ret += np.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret 

In [21]:
def eval(train_user_items, test_user_items, W, N):
    p = Precision(train_user_items, test_user_items, W, N)
    r = Recall(train_user_items, test_user_items, W, N)
    c = Coverage(train_user_items, test_user_items, W, N)
    pop = Popularity(train_user_items, test_user_items, W, N)
    return (p, r, c, pop)

In [22]:
print(eval(train_user_items, test_user_items, W, 5))
print(eval(train_user_items, test_user_items, W, 10))
print(eval(train_user_items, test_user_items, W, 20))
print(eval(train_user_items, test_user_items, W, 30))
print(eval(train_user_items, test_user_items, W, 40))
print(eval(train_user_items, test_user_items, W, 50))

(0.0696, 0.11607738492328219, 0.5958762886597938, 1.4077830628262402)
(0.0716, 0.11941294196130754, 0.6793814432989691, 1.414626173346446)
(0.0888, 0.14809873248832556, 0.7061855670103093, 1.4135327850250619)
(0.0928, 0.15476984656437626, 0.7587628865979381, 1.4249492106712118)
(0.0984, 0.16410940627084722, 0.7721649484536083, 1.4447690762703433)
(0.1004, 0.16744496330887257, 0.8164948453608247, 1.431797264092608)


In [ ]:
(0.0588, 0.0980653769179453, 0.6896907216494845, 1.3641942871593957)
(0.066, 0.11007338225483655, 0.7536082474226804, 1.33054347042702)
(0.0684, 0.11407605070046697, 0.7175257731958763, 1.3023239552299348)
(0.0712, 0.11874583055370247, 0.7164948453608248, 1.2930398697498386)
(0.0732, 0.12208138759172782, 0.7453608247422681, 1.26533467745946)
(0.0716, 0.11941294196130754, 0.7505154639175258, 1.2386142532419655)